In [8]:
import requests

def chat_with_ollama(prompt, model="llama3.2:latest"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # disable streaming
    }
    response = requests.post(url, json=payload)
    data = response.json()
    return data["response"]

if __name__ == "__main__":
    user_input = "Explain me string theory in simple terms."
    reply = chat_with_ollama(user_input)
    print(f"Ollama: {reply}")

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12c2d5880>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
import requests
import json

def chat_with_ollama(prompt, model="llama3.2:latest"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": True  # enable streaming
    }

    with requests.post(url, json=payload, stream=True) as resp:
        for line in resp.iter_lines():
            if line:  # skip empty lines
                data = json.loads(line.decode("utf-8"))
                if "response" in data:
                    yield data["response"]
                if data.get("done", False):
                    break

if __name__ == "__main__":
    user_input = "Tell me about Golden Cross in trading."
    print("Ollama:", end=" ", flush=True)
    for chunk in chat_with_ollama(user_input):
        print(chunk, end="", flush=True)
    print()

Ollama: The Golden Cross is a technical analysis indicator used to identify the potential start of a new uptrend in a stock's price. It occurs when the 50-day moving average (MA) crosses above the 200-day MA, indicating that the shorter-term trend has shifted into the long-term trend.

Here are some key aspects of the Golden Cross:

1. **Signal generation**: The Golden Cross is generated when the 50-day MA crosses above the 200-day MA.
2. **Confirmation**: A buy signal is confirmed when the stock price closes above its 50-day MA after the cross-over.
3. **Mean reversion**: Some traders believe that a Golden Cross can also be a sign of mean reversion, as it indicates that the short-term trend may reverse and become more aligned with the long-term trend.
4. **Uptrend confirmation**: The Golden Cross is considered a bullish indicator, confirming an uptrend and indicating that the stock's price is likely to continue rising.

**When to use:**

1. **Buy signals**: Use the Golden Cross as a b

In [ ]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# 1. Define your LLM (Ollama backend)
llm = Ollama(model="llama3.2:latest", base_url="http://localhost:11434")  
# If running in K8s with ingress: base_url="http://your-ollama-service"

# 2. Chain 1: Generate outline
prompt1 = PromptTemplate(
    input_variables=["subject"],
    template="Create a 3-point outline for a blog on {subject}."
)
chain1 = LLMChain(llm=llm, prompt=prompt1)

# 3. Chain 2: Expand outline into a blog
prompt2 = PromptTemplate(
    input_variables=["outline"],
    template="Write a blog article based on this outline:\n{outline}"
)
chain2 = LLMChain(llm=llm, prompt=prompt2)

# 4. Combine into sequential chain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# 5. Run the chain
print(overall_chain.invoke("Kubernetes autoscaling"))

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_32212/2714210623.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2:latest", base_url="http://localhost:11434")
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_32212/2714210623.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain1 = LLMChain(llm=llm, prompt=prompt1)




> Entering new SimpleSequentialChain chain...
Here is a 3-point outline for a blog on Kubernetes autoscaling:

I. Introduction to Kubernetes Autoscaling

* Brief overview of Kubernetes and its benefits
* Explanation of what Kubernetes autoscaling is and its importance in modern cloud-native applications
* Discussion of the challenges and limitations of traditional scaling approaches (e.g. static scaling, dynamic scaling)

II. Types of Kubernetes Autoscaling Strategies

* Horizontal Pod Autoscaling (HPA): a popular approach to automatically scale the number of replicas based on CPU utilization or other metrics
* Vertical Pod Autoscaling: a strategy for dynamically adjusting resources allocated to individual pods based on their workload needs
* Batch-based Autoscaling: an approach that scales applications in batches, often used for applications with predictable and recurring workloads

III. Best Practices for Implementing Kubernetes Autoscaling

* Overview of key considerations when im

In [9]:
# Trading Strategy: Volume Surge + 30-Day Low Reversal
import yfinance as yf
import pandas as pd
import datetime

# === Config ===
TICKERS = ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GNFC.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GPPL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JUSTDIAL.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KNRCON.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PEL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RAYMOND.NS', 'RAYMONDLSL.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RENUKA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'ROUTE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RTNINDIA.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SWSOLAR.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TANLA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS']
LOOKBACK = 30
START = (datetime.date.today() - datetime.timedelta(days=60)).strftime("%Y-%m-%d")
END = datetime.date.today().strftime("%Y-%m-%d")

results = []

for ticker in TICKERS:
    try:
        df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)

        if len(df) < LOOKBACK + 2:
            continue  # Not enough data

        df["30d_low"] = df["Low"].rolling(LOOKBACK).min()
        df["30d_high"] = df["High"].rolling(LOOKBACK).max()

        # Take last 2 rows (Yesterday, Today)
        today = df.iloc[-1]
        yesterday = df.iloc[-2]

        cond1 = (yesterday["Low"] == df["Low"].iloc[-LOOKBACK-1:-1].min())  # Yesterday's low = lowest of last 30
        cond2 = (today["Volume"] > 1.8 * yesterday["Volume"])              # Volume surge
        cond3 = (today["Low"] > yesterday["Low"])                         # Higher low
        cond4 = (today["Open"] > 20)                                      # Price filter
        cond5 = (today["High"] < df["High"].iloc[-LOOKBACK:].max())       # Not 30d breakout

        if cond1 and cond2 and cond3 and cond4 and cond5:
            results.append({
                "Ticker": ticker,
                "Yesterday_Low": yesterday["Low"],
                "Today_Low": today["Low"],
                "Today_High": today["High"],
                "Today_Volume": today["Volume"],
                "Yesterday_Volume": yesterday["Volume"]
            })
    except Exception as e:
        print(f"Error with {ticker}: {e}")

# Final Output
if results:
    res_df = pd.DataFrame(results)
    print("Stocks satisfying strategy:\n")
    display(res_df)
else:
    print("No stocks satisfy the strategy today.")


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_24041/445963484.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_24041/445963484.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_24041/445963484.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_24041/445963484.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False, multi_level_index=False)


Stocks satisfying strategy:



,Ticker,Yesterday_Low,Today_Low,Today_High,Today_Volume,Yesterday_Volume
0,NUVAMA.NS,6155.0,6159.0,6520.0,355864.0,184611.0


In [25]:
# Step 1: Import necessary libraries
import yfinance as yf
import pandas as pd
import talib as ta
from datetime import datetime
from IPython.display import display

print("Libraries imported successfully.")

# ==============================================================================
# Step 2: All Strategy Parameters
# ==============================================================================

# -- Ticker and Date Settings --
TICKERS_TO_SCAN = [
    'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS'
]
MARKET_INDEX = '^NSEI' # Use '^NSEI' for Nifty 50, '^GSPC' for S&P 500
START_DATE = "2010-01-01"
END_DATE = datetime.now().strftime('%Y-%-m-%d')

# -- Indicator Settings --
ST_ATR_PERIOD = 10
ST_MULTIPLIER = 3.0
VOL_MA_PERIOD = 20
MARKET_MA_PERIOD = 50

# -- Exit Strategy Settings --
SL_PERCENTAGE = 0.02  # Represents 2%

# ==============================================================================
# Step 3: Feature Toggles (Enable/Disable Rules Here)
# ==============================================================================
# Set a rule to 'True' to enable it, or 'False' to disable it.

ENABLE_SUPERTREND_FLIP = False    # Core Signal: Confirms a trend change just happened.
ENABLE_R1_S1_BREAK = True        # Confirmation: Price broke a key pivot level.
ENABLE_MOMENTUM_CANDLE = False    # Confirmation: The candle shows strong momentum (close > open).
ENABLE_VOLUME_CONFIRMATION = False# Confirmation: Volume is above average.
ENABLE_MARKET_CONTEXT = False     # Filter: Only trade in the direction of the broader market.

# ==============================================================================
# Step 4: Helper Function to Calculate Supertrend (No Changes Here)
# ==============================================================================
def calculate_supertrend(df, atr_period, multiplier):
    high, low, close = df['High'], df['Low'], df['Close']
    atr = ta.ATR(high, low, close, timeperiod=atr_period)
    
    supertrend_line, final_upper, final_lower = ([0.0] * len(df) for _ in range(3))
    
    for i in range(1, len(df)):
        upper_band = ((high.iloc[i] + low.iloc[i]) / 2) + (multiplier * atr.iloc[i])
        lower_band = ((high.iloc[i] + low.iloc[i]) / 2) - (multiplier * atr.iloc[i])

        final_upper[i] = upper_band if upper_band < final_upper[i-1] or close.iloc[i-1] > final_upper[i-1] else final_upper[i-1]
        final_lower[i] = lower_band if lower_band > final_lower[i-1] or close.iloc[i-1] < final_lower[i-1] else final_lower[i-1]
            
        if supertrend_line[i-1] == final_upper[i-1] and close.iloc[i] <= final_upper[i]:
            supertrend_line[i] = final_upper[i]
        elif supertrend_line[i-1] == final_upper[i-1] and close.iloc[i] > final_upper[i]:
            supertrend_line[i] = final_lower[i]
        elif supertrend_line[i-1] == final_lower[i-1] and close.iloc[i] >= final_lower[i]:
            supertrend_line[i] = final_lower[i]
        elif supertrend_line[i-1] == final_lower[i-1] and close.iloc[i] < final_lower[i]:
            supertrend_line[i] = final_upper[i]

    df['Supertrend_Line'] = supertrend_line
    df['Supertrend_Direction'] = (close > df['Supertrend_Line']).astype(int).replace(0, -1)
    return df

# ==============================================================================
# Step 5: Main Scanner Logic
# ==============================================================================
print("Starting trading strategy scanner...")

# --- Determine Broader Market Context ---
is_market_bullish = True # Default to true if context is disabled or fails
try:
    if ENABLE_MARKET_CONTEXT:
        index_df = yf.download(MARKET_INDEX, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)
        index_df['MA'] = index_df['Close'].rolling(window=MARKET_MA_PERIOD).mean()
        is_market_bullish = index_df.iloc[-1]['Close'] > index_df.iloc[-1]['MA']
        market_status = "Bullish" if is_market_bullish else "Bearish"
        print(f"Market Context ({MARKET_INDEX}): {market_status}")
    else:
        print("Market Context: Disabled")
except Exception:
    print(f"Could not determine market context. Defaulting to allow all trades.")

final_signals = []
print(f"Scanning {len(TICKERS_TO_SCAN)} tickers...")

for ticker in TICKERS_TO_SCAN:
    try:
        df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)
        if len(df) < MARKET_MA_PERIOD: continue

        # --- Calculate Indicators ---
        df = calculate_supertrend(df, ST_ATR_PERIOD, ST_MULTIPLIER)
        df['Volume_MA'] = df['Volume'].rolling(window=VOL_MA_PERIOD).mean()
        df['PP'] = (df['High'].shift(1) + df['Low'].shift(1) + df['Close'].shift(1)) / 3
        df['R1'] = (2 * df['PP']) - df['Low'].shift(1)
        df['R2'] = df['PP'] + (df['High'].shift(1) - df['Low'].shift(1))
        df['S1'] = (2 * df['PP']) - df['High'].shift(1)
        df['S2'] = df['PP'] - (df['High'].shift(1) - df['Low'].shift(1))
        df.dropna(inplace=True)
        if df.empty: continue
        
        latest = df.iloc[-1]
        prev = df.iloc[-2]

        # ==============================================================================
        # MODIFIED SECTION: Apply Entry Rules Based on Toggles
        # ==============================================================================
        signal = "No Signal"

        # --- Define Long Conditions ---
        long_cond_st_flip = prev['Supertrend_Direction'] == -1 and latest['Supertrend_Direction'] == 1
        long_cond_breakout = latest['Close'] > latest['R1']
        long_cond_candle = latest['Close'] > latest['Open']
        long_cond_volume = latest['Volume'] > latest['Volume_MA']
        long_cond_market = is_market_bullish

        if (not ENABLE_SUPERTREND_FLIP or long_cond_st_flip) and \
           (not ENABLE_R1_S1_BREAK or long_cond_breakout) and \
           (not ENABLE_MOMENTUM_CANDLE or long_cond_candle) and \
           (not ENABLE_VOLUME_CONFIRMATION or long_cond_volume) and \
           (not ENABLE_MARKET_CONTEXT or long_cond_market):
            signal = "Long"

        # --- Define Short Conditions ---
        short_cond_st_flip = prev['Supertrend_Direction'] == 1 and latest['Supertrend_Direction'] == -1
        short_cond_breakdown = latest['Close'] < latest['S1']
        short_cond_candle = latest['Close'] < latest['Open']
        short_cond_volume = latest['Volume'] > latest['Volume_MA']
        short_cond_market = not is_market_bullish
        
        # Only check for a short signal if a long one wasn't already found
        if signal == "No Signal":
            if (not ENABLE_SUPERTREND_FLIP or short_cond_st_flip) and \
               (not ENABLE_R1_S1_BREAK or short_cond_breakdown) and \
               (not ENABLE_MOMENTUM_CANDLE or short_cond_candle) and \
               (not ENABLE_VOLUME_CONFIRMATION or short_cond_volume) and \
               (not ENABLE_MARKET_CONTEXT or short_cond_market):
                signal = "Short"
        
        # --- Store Signal and Calculate Exits ---
        if signal != "No Signal":
            stop_loss = latest['Supertrend_Line'] * (1 - SL_PERCENTAGE) if signal == "Long" else latest['Supertrend_Line'] * (1 + SL_PERCENTAGE)
            profit_target = latest['R2'] if signal == "Long" else latest['S2']

            final_signals.append({
                'Date': latest.name.strftime('%Y-%-m-%d'), 'Ticker': ticker, 'Signal': signal,
                'Entry_Price': f"{latest['Close']:.2f}", 'Stop_Loss': f"{stop_loss:.2f}",
                'Profit_Target': f"{profit_target:.2f}"
            })
            print(f"--- Signal Found for {ticker}: {signal} ---")

    except Exception as e:
        print(f"Could not analyze {ticker}. Reason: {e}")

# ==============================================================================
# Step 6: Output Results
# ==============================================================================
print("\n--- Scan Complete ---")

if final_signals:
    signals_df = pd.DataFrame(final_signals)
    output_filename = f'trading_signals_{datetime.now().strftime("%Y-%m-%d")}.csv'
    signals_df.to_csv(output_filename, index=False)
    
    print(f"\nSuccessfully generated {len(signals_df)} signals.")
    print(f"Results saved to '{output_filename}'")
    display(signals_df)
else:
    print(f"\nNo trading signals found for the given tickers and enabled rules.")

Libraries imported successfully.
Starting trading strategy scanner...
Market Context: Disabled
Scanning 4 tickers...


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_26144/457545484.py:95: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_26144/457545484.py:95: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_26144/457545484.py:95: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)


--- Signal Found for HDFCBANK.NS: Long ---
--- Signal Found for INFY.NS: Long ---

--- Scan Complete ---

Successfully generated 2 signals.
Results saved to 'trading_signals_2025-09-19.csv'


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_26144/457545484.py:95: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, multi_level_index=False)


,Date,Ticker,Signal,Entry_Price,Stop_Loss,Profit_Target
0,2025-9-18,HDFCBANK.NS,Long,976.90,0.00,978.47
1,2025-9-18,INFY.NS,Long,1540.60,0.00,1533.73
